## Thinking1 什么是近似最近邻查找，常用的方法有哪些

##### 近似最近邻查找是在最近邻查找问题基础上来说的。NN，最近邻查找，其中的k最近邻，就是查找离目标数据最近的前k个数据项。ANN，近似最近邻检索，在牺牲可接受范围内的精度的情况下提高检索效率。

在数据维度较低且数据量较小的情况下，可以使用最近邻方法计算。

然而针对海量数据且数据维度较高的情况下，使用最近邻方法时间和空间成本太高，效率太低。因此需要降维，采用索引方式查找。

ANN中常用的方法有 LSH局部敏感哈希。LSH将相邻的数据，通过映射后依然保持相邻的关系，即保持局部的敏感度。其中LSH分别有MinHashLSH, MinHashLSHForest, MinHashLSHEnsemble等几种不同的算法。

## Thinking2 为什么两个集合的minhash值相同的概率等于这两个集合的Jaccard相似度

##### 将两个集合（比如a和b集合）进行0-1编码。对应位置上ai和bi共有三种情况。

##### A情况，ai和bi的值都为1
B情况，其中一列的值为0，另一列的值为1

C情况，两列的值都为0

##### 根据Jaccard相似度的定义可知，某两个集合的交集除以这两个集合的并集 即  a/(a+b)

##### 根据minhash的定义（特征矩阵按行进行随机排列后，第一个列值为1的行的行号）以及概率论中排列组合的知识可知，实际上C情况对于两个集合的minhash值相同概率没有影响。

##### 这里为何minhash与C情况无关，个人的理解是（稍微直白点说，不管C情况在A和B出现之前出现过多少次，影响的只是两个集合的minhash值大小，不影响两个集合的minhash相等的可能，因为两个集合的minhash是否相等,决定权在A和B中，C情况出现只能让这两个集合的minhash值增大。如果A先于B出现，那么这两个集合的minhash值相等，否则，这两个集合的minhash值不同。）

##### 因此两个集合的minhash值相等的情况有a种，不等的情况有b种，因此概率亦为a/(a+b)

## Thinking3 SimHash在计算文档相似度的作用是怎样的？

##### 文档指纹计算过程

Step1，设置SimHash的位数，比如32位，设置位数需要考虑存储成本以及数据集的大小。

Step2，初始化SimHash，将各位初始化为0。

Step3，使用N-gram方法提取文本中的特征。

Step4，使用传统的hash函数计算各个word的hashcode。

Step5，对各word的hashcode的每一位，如果该位为1，则simhash相应位的值加它的权重（通常是出现的频率）；否则减它的权重。

Step6，计算最后得到的32位的SimHash，如果该位大于1，则设为1；否则设为0。

通过以上几步，使用SimHash算法得到了每篇文档的指纹（fingerprint）。

##### 汉明距离计算

可以使用汉明（Hamming）距离计算文档之间的相似度。汉明（Hamming）距离的定义是：两个二进制串中不同位的数量。向量相似度越高，对应的汉明距离越小。

##### 通过文档的SimHash计算文档之间的相似度

通过SimHash算法可得到每篇文档的指纹（fingerprint）。然后计算两个文档指纹的海明距离。通常2篇文档的Hamming距离在3以内，就认为相似度比较高。

直接对海量文档比较文档指纹开销较大。采用另一种方法，即 如果汉明距离为3以内相似度较高，就可以将指纹分为4段。如果有一段相同，那么文档相似的可能性就很大，可以通过这个方法，降低计算成本。

## Thinking4 为什么YouTube采用期望观看时间作为评估指标


##### CTR指标对于视频搜索具有一定的欺骗性，比如用户点击了一个视频，但是发现他并不喜欢这类视频，马上就关

闭了视频，然而在CTR预估中，会将用户点击的视频分为正样本，这样会对用户个性化推荐产生误导，

甚至降低用户体验，降低客户忠诚度。所以youtube提出采用期望观看时间作为评估指标,

划分样本空间时，正样本为点击视频，输出值即阅读时长值；

负样本为无点击视频，输出值则统一采用1，即采用单位权值，不进行加权。

这种评估指标在视频推荐系统上效果较经典的CTR，或者Play Rate好一些。



### Thinking5 为什么YouTube在排序阶段没有采用经典的LR（逻辑回归）当作输出层，而是采用了Weighted Logistic Regression？


##### 也可以使用经典的LR，但是由于CTR指标对于视频搜索具有一定的欺骗性，因此引入了观看时长的计算。观看时长不是只有0，1两种标签，在视频行业，一个用户观看时间较长，可以认为这个用户的商业化价值较高，相比较点击率的计算更加准确。所以YouTube采用了Weighted Logistic Regression来模拟这个输出。

即划分样本空间时，正样本用观看时间赋予权值，负样本赋予单位权值。


### Action1 "使用MinHashLSHForest对微博新闻句子进行检索 weibo.txt  针对某句话进行Query，查找Top-3相似的句子"



#### 具体代码请见 MinHashLSHForest.ipynb 和 MinHashLSHForest2.ipynb 文件

##### 其中 MinHashLSHForest.ipynb 没有去掉查询的那一句，因此有一项相似程度为1。而 MinHashLSHForest2.ipynb 去掉了查询的那一句，结果稍有不同。

##### 代码实现基本和参考代码差不多

源代码中这一句会报错  stop = [line.strip().decode('utf-8') for line in open('stopword.txt').readlines()]

错误类型：AttributeError: 'str' object has no attribute 'decode'

改为这句就没错误了 stop = [line.strip().encode('utf-8') for line in open('stopword.txt').readlines()]

同时也设置了一个停用词文件，比如一些标点符号等等。

### Action2 请设计一个基于DNN模型的推荐系统

阐述两阶段的架构（召回、排序）

以及每个阶段的DNN模型设计：

1) DNN输入层（如何进行特征选择）

2) DNN隐藏层结构

3) DNN输出层


### 整体架构


##### 这两个阶段的架构比较类似，都是DNN模型。

均由DNN输入层，DNN隐藏层，DNN输出层组成。

输入层为各种特征通过embedding方法生成的输入向量。只不过两阶段特征的粒度不同，排序阶段较为细致一些。

架构中间是三层隐藏层的DNN结构。使用ReLU作为激活函数。

输出层也类似，只不过完成的功能不同，略有差别。

### DNN输入层

#### 召回阶段


#### 对输入的特征进行embedding：

用类似于word2vec的做法，每个视频都会被embedding到固定维度的向量中。最终通过加权平均（可根据重要性和时间进行加权）得到固定维度的watch vector。Search vector，和watch vector生成方式类似。用户画像特征：如地理位置，设备，性别，年龄，登录状态等连续或离散特征都被归一化，和watch vector以及search vector做拼接（concatenate）。还有添加特征——样本年龄等。

小技巧：

使用更广泛的数据源，防止面对新视频的时候很难推荐，并且推荐器会过度偏向exploitation。

为每个用户生产固定数量的训练样本，防止一部分非常活跃的用户主导损失函数值。

负采样问题，随机从全量item中抽取用户没有点击过的item作为label=0的item。

以及不对称的共同浏览问题。


#### 排序阶段



##### 对输入的特征进行embedding

排序阶段在召回阶段的基础上，可以引入更多的feature（当前要计算的video的embedding，用户观看过的最后N个视频embedding的average，用户语言的embedding和当前视频语言的embedding，自上次观看同channel视频的时间，该视频已经被曝光给该用户的次数）。粒度相对召回阶段应该更细致，因为召回阶段是粗略召回相似的商品，而排序要求较高。

### DNN隐藏层结构

##### 两阶段的设计在隐藏层基本相同。

采用的Tower塔式模型，例如第一层1024，第二层512，第三层256，使用ReLU作为激活函数。

### DNN输出层

##### 召回阶段

Training 阶段输出层为softmax层，
Serving 阶段直接用user Embedding和video Embedding计算dot-product表示分数，取topk作为候选结果。

最重要问题是在性能。因此使用类似局部敏感哈希LSH（近似最近邻方法）

##### 排序阶段

Training最后一层是Weighted LR，Serving时激励函数使用的e^(w*x+b)。
